## *Rodar notebook toda sexta-feira já que às quintas os atletas têm suas habilidades atualizadas após os treinos e os jogos da semana*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db
from datetime import datetime

In [2]:
# Mantendo o padrão de como o sokker mostra as datas nos dados que disponibiliza
agora= str(datetime.now()).split('.')[0]

print("Data:", agora)

Data: 2021-07-27 14:44:35


In [3]:
sessao, club_id= clube_id_e_sessao()

Login: fabin99
Password: ········
Clube ID: 79085


Como meu clube não pode ter valores nulos, os dos outros usuários sim (não tenho acesso a alguns dados de seus jogadores), eu insiro uma cláusula if antes de iniciar a inserção dos dados na base de dados para conferir.

In [4]:
# URL da API
url= f'https://sokker.org/xml/players-{club_id}.xml'

# Pegando os dados dos atletas
df_players= pega_dados(url, sessao, search= 'player')

# Inserindo a data de hoje que será importante para acompanharmos suas evoluções
df_players= df_players.assign(date_extracted= agora)

# Soma de valores nulos ou duplicados
nulos= df_players.isnull().sum().sum()
duplicados=  df_players.duplicated().sum()


if (nulos > 0) or (duplicados > 0):
    print('Há valores nulos ou duplicados. Confira os dados e tente novamente.')
    
else:
    print('Não há valores nulos ou duplicados. Inserindo dados na base de dados "dados.sokker.db"...')
    
    db= conecta_ao_db('dados_sokker.db')
    curs= db.cursor()
    
    df_players.to_sql('players', con=db, if_exists='append', index= False) 
    print('Dados extraídos com sucesso!')

Não há valores nulos ou duplicados. Inserindo dados na base de dados "dados.sokker.db"...
Dados extraídos com sucesso!


In [5]:
df = pd.read_sql_query("SELECT * FROM players", db)

In [6]:
df

,ID,date_extracted,name,surname,countryID,age,teamID,wage,height,weight,...,skillStamina,skillTeamwork,skillDiscipline,skillPace,skillTechnique,skillPassing,skillKeeper,skillDefending,skillPlaymaking,skillScoring
0,36519432,2021-07-27 14:44:35,Krastan,Apostolov,54,29,79085,33900.0,177.0,71.5,...,9,1,5,15,3,9,10,5,5,3
1,37968035,2021-07-27 14:44:35,Romildo,Barbadiano,21,29,79085,3900.0,171.0,63.9,...,0,1,1,5,8,3,0,8,1,3
2,37968034,2021-07-27 14:44:35,Giovani,Barriguinha,21,27,79085,2600.0,172.0,76.3,...,3,1,0,6,2,5,1,5,1,4
3,36103236,2021-07-27 14:44:35,Ilario,Bartolozzi,10,29,79085,16900.0,169.0,67.3,...,11,1,9,13,7,8,0,8,9,8
4,37968028,2021-07-27 14:44:35,Cecilio,Bazzarella,21,20,79085,4800.0,175.0,77.4,...,1,2,1,7,1,9,0,5,7,7
5,37968048,2021-07-27 14:44:35,Josimar,Bendler,21,20,79085,4700.0,166.0,66.9,...,0,2,1,7,2,7,7,1,1,0
6,37937282,2021-07-27 14:44:35,Mustapha,Bonnet,16,18,79085,4700.0,183.0,72.3,...,2,0,0,10,3,0,0,6,0,7
7,37464066,2021-07-27 14:44:35,Rody,de Houston,21,22,79085,16300.0,173.0,74.9,...,6,0,2,12,2,9,10,2,3,3
8,37432229,2021-07-27 14:44:35,Christophe,Derungs,24,22,79085,7200.0,183.0,91.3,...,11,0,1,10,8,9,0,7,2,2
9,37442584,2021-07-27 14:44:35,Fabricio,Fontanello,19,22,79085,15200.0,166.0,61.2,...,8,2,3,11,7,11,8,7,4,4


In [7]:
len(df)

27

In [8]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()